In [1]:
import fasttext
import numpy as np
import sys
sys.path.append('..')

In [2]:
import logging
log_format = '[%(asctime)s] [%(levelname)s] - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_format)
log = logging.getLogger(__name__)

# writing to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
handler.setFormatter(log_format)
log.addHandler(handler)
logging.debug('hello')

[2020-01-16 23:29:02,624] [DEBUG] - hello


In [3]:
model = fasttext.load_model('/home/legin/fasttext/wiki.en.bin')

In [6]:
m = model.f.getInputMatrix()

In [15]:
del m

In [7]:
import compressors.compressor8 as d 
import storages.dict_based as db 
from importlib import reload
reload(d)

<module 'compressors.compressor8' from '/home/legin/ft_compress/ft_compress/compressors/compressor8.py'>

In [8]:
dc = d.Compressor8bit(db.DictStorage()) #~50% percents

In [9]:
dc.fit(model, 10)

[2020-01-16 23:32:26,871] [DEBUG] - loading words
[2020-01-16 23:32:31,445] [DEBUG] - Estimating parameters
[2020-01-16 23:32:48,928] [DEBUG] - filling matrix
[2020-01-16 23:33:19,027] [DEBUG] - ready


In [10]:
dc.info()

'Number of ngrams: 1868814. Approximate size: 544.8786640167236 MB'

In [12]:
dc['h'] - model['h']

[2020-01-16 23:33:47,016] [DEBUG] - 1:1:h


array([-3.27280003e-01, -3.33854441e-01,  2.68502501e-01,  2.08985434e-01,
        1.92106766e-01, -9.54895444e-02, -8.07417234e-03, -2.69404634e-01,
       -5.92559028e-01, -2.58542254e-01,  1.45860163e-01, -4.28805949e-01,
       -1.82555059e-01, -2.64012799e-01, -7.62753872e-02, -2.50889022e-01,
       -5.45006233e-02, -8.92976949e-03,  1.31687526e-01,  1.52560716e-01,
        3.76739101e-01,  1.64215130e-02, -2.81351059e-01,  6.84964720e-02,
       -4.08244087e-01, -5.80970358e-01,  2.30231338e-02, -4.29149572e-01,
       -1.18356711e-01,  1.77450152e-01, -1.27146194e-01,  5.37015251e-02,
       -1.06537179e-01, -2.68550842e-01, -1.84558971e-01, -3.19721576e-02,
        6.69967962e-02,  1.84612710e-01, -8.63754845e-02, -1.99336820e-02,
        1.21276377e-01, -1.21460556e-01,  5.46953170e-02, -2.25250154e-01,
       -1.20209740e-02, -4.25623343e-02, -7.54834807e-02,  4.06850862e-01,
       -8.92625761e-02,  2.99888492e-02,  2.08634238e-01, -6.28287392e-01,
       -2.30025690e-01, -

In [9]:
import compressors.compressor8 as d 
from importlib import reload
reload(d)

<module 'compressors.compressor8' from '/home/legin/ft_compress/ft_compress/compressors/compressor8.py'>

In [10]:
dc = d.Compressor8bit(model, 2500000) #~50% percents

[2020-01-12 14:35:37,861] [DEBUG] - Loading words
[2020-01-12 14:35:48,702] [WARNING] - Too many ngrams, breaking
[2020-01-12 14:35:48,702] [WARNING] - Too many ngrams, breaking
[2020-01-12 14:35:48,702] [DEBUG] - Estimating parameters
[2020-01-12 14:36:15,707] [DEBUG] - filling matrix


KeyboardInterrupt: 

In [23]:
model.get_subwords('bananananana')[0]

['<ba',
 '<ban',
 '<bana',
 '<banan',
 'ban',
 'bana',
 'banan',
 'banana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'ananan',
 'nan',
 'nana',
 'nanan',
 'nanana',
 'ana',
 'anan',
 'anana',
 'anana>',
 'nan',
 'nana',
 'nana>',
 'ana',
 'ana>',
 'na>']

In [0]:
v = model.get_input_vector(0)

In [0]:
v2 = np.round(255*(v-dc.min_)/(dc.max_-dc.min_)).astype(np.uint8)


In [0]:
(dc.bytes_to_vec(dc.ngrams[',']) - v).sum()

In [0]:
dc.max_, v.max()

In [0]:
dc.ngrams

In [0]:
dc.info()